In [9]:
import requests
from bs4 import BeautifulSoup
import csv
import warnings


warnings.filterwarnings('ignore', message='Unverified HTTPS request')

def get_project_details(project_url, headers):
    response = requests.get(project_url, headers=headers, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')

    details = {}


    details['GSTIN No'] = soup.find(text='GSTIN No').find_next('td').text.strip()
    details['PAN No'] = soup.find(text='PAN No').find_next('td').text.strip()
    details['Name'] = soup.find(text='Promoter Name').find_next('td').text.strip()
    details['Permanent Address'] = soup.find(text='Address').find_next('td').text.strip()

    return details

def scrape_projects(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(url, headers=headers, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')

    projects = []


    project_links = soup.select('a[href*="RERAPROJECTID"]')[:6]

    for link in project_links:
        project_url = 'https://hprera.nic.in' + link.get('href')
        project_details = get_project_details(project_url, headers)
        projects.append(project_details)

    return projects


url = 'https://hprera.nic.in/PublicDashboard'
projects = scrape_projects(url)


csv_file = 'projects_details.csv'


with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['GSTIN No', 'PAN No', 'Name', 'Permanent Address'])
    writer.writeheader()
    for project in projects:
        writer.writerow(project)

print(f"Project details have been written to {csv_file}")


for i, project in enumerate(projects):
    print(f"Project {i+1}:")
    for key, value in project.items():
        print(f"{key}: {value}")
    print()


Project details have been written to projects_details.csv
